Importar dependencias

In [ ]:
import pandas as pd
import tensorflow as tf
import keras_cv
from keras_cv import bounding_box
from keras_cv.models import RetinaNet
import os

Cargar archivo annotations.csv y clases a utilizar

In [ ]:
# Leer anotaciones
df = pd.read_csv("annotations.csv")
image_dir = tf.constant("/images") # porque está esperando un tensor

# Mapeo de clases a IDs
class_mapping = {'monster': 0, 'spell': 1, 'trap': 2}

Realizar proceso se data augmentation, para multiplicar artificialmente la diversidad del set

In [ ]:

def load_data(row):
    # Obtener ruta completa como tensor
    img_path = tf.strings.join([image_dir, row['filename']], separator="/")
    
    # Leer y decodificar imagen
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  # o decode_png si son PNG

    # Redimensionar y normalizar
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0

    # Extraer cajas y etiquetas (si están en el DataFrame)
    boxes = row['boxes']  # Debe ser tensor (num_boxes, 4)
    classes = row['classes']  # Debe ser tensor (num_boxes,)

    return {
        "images": img,
        "bounding_boxes": {
            "classes": classes,
            "boxes": boxes
        }
    }

# Convertir a tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.map(load_data).batch(8)

# Usar augmentation de KerasCV (opcional)
augmenter = keras_cv.layers.Augmenter([
    keras_cv.layers.RandomFlip(mode="horizontal"),
    keras_cv.layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),
])

dataset = dataset.map(lambda x: augmenter(x))

Mostrar ejemplo

In [ ]:
for batch in dataset.take(1):
    keras_cv.visualization.plot_bounding_box_gallery(
        images=batch["images"],
        value_range=(0, 255),
        bounding_boxes=batch["bounding_boxes"],
        rows=1,
        cols=1,
        class_mapping=class_mapping,
    )